In [1]:
from urllib.request import urlopen, Request
from urllib.request import urlopen
import datetime
from time import sleep
from bs4 import BeautifulSoup
import copy
import json
import sys
import csv
import pandas as pd
import numpy as np




# Link Crawl section

In [13]:
page = urlopen("file:/C:/Users/PeteK/Desktop/School/DPNewsScripts/newsbtc_crawl/Breaking News on Bitcoin, Price Prediction and Analysis.html") 
result=[]
soup = BeautifulSoup(page.read(), 'lxml', from_encoding="unicode")
hrefs = soup.find_all('a', class_='featured-image effect-zoom')
for href in hrefs:
    result.append(href['href'])
#print(result)

urlsDf = pd.DataFrame(result, columns =['urls'])

In [15]:
urlsDf.to_csv("newsbtclinks.csv", encoding='utf-8', index=False)

# Content Crawl

#### Work with soup
soup.find_all('a', class_='featured-image')  
href['href'] - where href is result of find('a')  
soup.find(id="link3")
_tag.text - jadro tagu  
_tag.next_sibling - text dalsieho tagu

In [50]:
def CheckValidUrl(url):
    if "https://" not in url:
            print("I am skipping wrong html")
            return False
    return True
        
def GetUrlSoup(url):
    headers = { 'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)' }
    req = Request(url=url, headers=headers) 
    page = urlopen(req)
    soup = BeautifulSoup(page.read(), 'lxml', from_encoding="unicode")
    return soup;

def GetAuthorOfPublication(soup):
    author = soup.find('span', class_="author")
    author = author.find('a').text 
    return author;

def GetDateOfPublish(soup):
    date = soup.find('meta', property="article:published_time")
    date = (date['content'])
    date = date.split('T')[0]
    print(date)
    date_time = datetime.datetime.strptime(date, '%Y-%m-%d')
    print(date_time)
    return date_time

def GetTimeOfPublish(soup):
    date = soup.find('meta', property="article:published_time")
    date = (date['content'])[:-6].replace("T", " ")
    date_time = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    print(date_time)
    return date_time

def GetHeader(soup):
    header = soup.find('h1', class_="title large").text
    return header.encode('ascii', 'ignore').decode("utf-8")

def GetSubHeaders(soup):
    subHeaders = []
    divContent = soup.find('h2')
    headers = divContent
    print(headers)
    for header in headers:
        print(header)
        try:
            subHeaders.append((header.text).encode('ascii', 'ignore').decode("utf-8"))
        except:
            subHeaders.append((header).encode('ascii', 'ignore').decode("utf-8"))
    return subHeaders
    
def GetContent(soup):
    TextContent = ""
    divContent = soup.find('div', class_="article-content")
    textElements = divContent.find_all('p')
    for p in textElements:
        if "You must be" in p.text:
            continue
        TextContent = TextContent +" "+ p.text
    return TextContent.encode('ascii', 'ignore').decode("utf-8")
    
def GetTags(soup):
    tags=[]
    sectionTags = soup.find('span', class_="tags")
    stream = sectionTags.find_all('a')
    for tag in stream:
        tags.append((tag.text).encode('ascii', 'ignore').decode("utf-8"))
    return tags
    

In [51]:
resultPd = pd.DataFrame(columns =['source','url','author','date','datetime','headertext','subheaders','contenttext','tags'])
with open('newsbtclinks.csv') as file:
    reader = csv.reader(file, delimiter=',')
    for row in reader:
        rowPd = pd.DataFrame(columns =['source','url','author','date','datetime','headertext','subheaders','contenttext','tags'])
        crawlurl=row[0]
        if not CheckValidUrl(crawlurl):
            continue;
        soup = GetUrlSoup(crawlurl)
        
        #content Crawl
        Source = "www.newsbtclinks.com";
        Url = crawlurl;
        print(Url)
        Author = GetAuthorOfPublication(soup);
        Date = GetDateOfPublish(soup);
        DateTime = GetTimeOfPublish(soup);
        HeaderText = GetHeader(soup);
        SubHeadersText = GetSubHeaders(soup);
        ContentText = GetContent(soup);
        Tags = GetTags(soup);
    
    
        rowPd.at[0,'source'] = Source
        rowPd.at[0,'url'] = Url
        rowPd.at[0,'author'] = Author
        rowPd.at[0,'date'] = Date
        rowPd.at[0,'datetime'] = DateTime
        rowPd.at[0,'headertext'] = HeaderText
        rowPd.at[0,'subheaders'] = SubHeadersText
        rowPd.at[0,'contenttext'] = ContentText
        rowPd.at[0,'tags'] = Tags
        print(Date)
        frames = [resultPd, rowPd]
        resultPd = pd.concat(frames)


I am skipping wrong html
https://www.newsbtc.com/2019/09/26/bitcoin-outperforming-2016s-recovery-price-crash-to-7700-not-bearish/
2019-09-26
2019-09-26 00:00:00
2019-09-26 18:30:30
<h2><strong>Bitcoin Price Recovery Still on Track</strong></h2>
<strong>Bitcoin Price Recovery Still on Track</strong>
2019-09-26 00:00:00
https://www.newsbtc.com/2019/09/26/crypto-analyst-under-5000-bitcoin-now-a-possibility/
2019-09-26
2019-09-26 00:00:00
2019-09-26 18:00:42
<h2>Could Bitcoin Price Return to Bear Market Lows?</h2>
Could Bitcoin Price Return to Bear Market Lows?
2019-09-26 00:00:00
https://www.newsbtc.com/2019/09/26/bitcoin-price-drop-to-8500-not-a-bad-thing-heres-why/
2019-09-26
2019-09-26 00:00:00
2019-09-26 17:00:28
<h2><strong>No Surprise Crypto is in Consolidation</strong></h2>
<strong>No Surprise Crypto is in Consolidation</strong>
2019-09-26 00:00:00
https://www.newsbtc.com/2019/09/26/bitcoin-price-breaks-below-8000-just-days-after-triangle-breakdown/
2019-09-26
2019-09-26 00:00:00
2

https://www.newsbtc.com/2019/09/23/ceo-explains-why-todays-bakkt-launch-is-an-important-milestone-for-bitcoin/
2019-09-23
2019-09-23 00:00:00
2019-09-23 20:00:40
<h2>What Can Bakkt Do For Bitcoin?</h2>
What Can Bakkt Do For Bitcoin?
2019-09-23 00:00:00
https://www.newsbtc.com/2019/09/23/inverted-bitcoin-price-chart-is-bear-market-bottom-flipped-upside-down/
2019-09-23
2019-09-23 00:00:00
2019-09-23 19:00:17
<h2>Inverse Bitcoin Price Chart Similar to Bear Market Bottom, Drop To Follow?</h2>
Inverse Bitcoin Price Chart Similar to Bear Market Bottom, Drop To Follow?
2019-09-23 00:00:00
https://www.newsbtc.com/2019/09/23/bitcoin-price-signal-technically-in-bear-territory-can-btc-recover/
2019-09-23
2019-09-23 00:00:00
2019-09-23 12:00:57
<h2><strong>Bitcoin Price Teetering on Edge of Bear Zone</strong></h2>
<strong>Bitcoin Price Teetering on Edge of Bear Zone</strong>
2019-09-23 00:00:00
https://www.newsbtc.com/2019/09/23/bakkt-volume-a-slow-build-for-bitcoin-btc-futures-trading/
2019-09-2

https://www.newsbtc.com/2019/09/19/bitcoin-plunges-450-in-minutes-loses-10000-support-again/
2019-09-19
2019-09-19 00:00:00
2019-09-19 03:13:47
<h2 class="title small">Related News</h2>
Related News
2019-09-19 00:00:00
https://www.newsbtc.com/2019/09/18/wells-fargo-crypto/
2019-09-18
2019-09-18 00:00:00
2019-09-18 22:00:07
<h2>It’s Really Not a Crypto Though is it?</h2>
It’s Really Not a Crypto Though is it?
2019-09-18 00:00:00
https://www.newsbtc.com/2019/09/18/bitcoin-etf-dreams-are-fantasies-for-now-but-does-it-matter-for-btc/
2019-09-18
2019-09-18 00:00:00
2019-09-18 21:00:55
<h2><strong>Bitcoin ETF Application Pulled as Approval Odds Dwindled</strong><strong> </strong></h2>
<strong>Bitcoin ETF Application Pulled as Approval Odds Dwindled</strong>
<strong> </strong>
2019-09-18 00:00:00
https://www.newsbtc.com/2019/09/18/crypto-wyckoff-distribution-bitcoin-accumulation/
2019-09-18
2019-09-18 00:00:00
2019-09-18 19:00:27
<h2>Wyckoff Method: Is Bitcoin in a Reaccumulation Phase or Is 

https://www.newsbtc.com/2019/09/13/poll-more-than-half-of-bitcoin-investors-expect-triangle-breakout/
2019-09-13
2019-09-13 00:00:00
2019-09-13 19:00:47
<h2 class="p1"><span class="s1">Poll: Nearly 60% of Crypto Traders Expect Bitcoin Formation to Break Up</span></h2>
<span class="s1">Poll: Nearly 60% of Crypto Traders Expect Bitcoin Formation to Break Up</span>
2019-09-13 00:00:00
https://www.newsbtc.com/2019/09/13/analyst-friday-the-13th-daily-close-could-set-future-bitcoin-price-trend/
2019-09-13
2019-09-13 00:00:00
2019-09-13 18:00:35
<h2 class="p1"><span class="s1">Bitcoin Price: Friday the 13th Daily Close Especially Significant </span></h2>
<span class="s1">Bitcoin Price: Friday the 13th Daily Close Especially Significant </span>
2019-09-13 00:00:00
https://www.newsbtc.com/2019/09/13/binance-bitcoin-futures-debuts-with-170m-trading/
2019-09-13
2019-09-13 00:00:00
2019-09-13 14:00:26
<h2>The Season of Bitcoin Futures</h2>
The Season of Bitcoin Futures
2019-09-13 00:00:00
https://

https://www.newsbtc.com/2019/09/10/bitcoin-stranded-as-sec-chief-indicates-behindhand-etf/
2019-09-10
2019-09-10 00:00:00
2019-09-10 13:00:49
<h2>Equal Odds at Play</h2>
Equal Odds at Play
2019-09-10 00:00:00
https://www.newsbtc.com/2019/09/10/bitcoin-price-could-go-sideways-warn-analysts-stuck-at-10000/
2019-09-10
2019-09-10 00:00:00
2019-09-10 12:00:50
<h2>Bitcoin Price to Flatline</h2>
Bitcoin Price to Flatline
2019-09-10 00:00:00
https://www.newsbtc.com/2019/09/10/will-more-bitcoin-futures-be-big-for-markets-or-are-they-bad-news/
2019-09-10
2019-09-10 00:00:00
2019-09-10 05:00:39
<h2>Bitcoin Futures Good or Bad?</h2>
Bitcoin Futures Good or Bad?
2019-09-10 00:00:00
https://www.newsbtc.com/2019/09/10/next-bitcoin-bull-market-could-take-years-so-how-long-to-next-peak/
2019-09-10
2019-09-10 00:00:00
2019-09-10 04:00:10
<h2>Bitcoin Could Take Its Time To Reach Next High</h2>
Bitcoin Could Take Its Time To Reach Next High
2019-09-10 00:00:00
https://www.newsbtc.com/2019/09/09/research-b

...

https://www.newsbtc.com/2019/04/14/slowing-macroeconomy-boon-bitcoin-say-researchers/
2019-04-14
2019-04-14 00:00:00
2019-04-14 00:00:16
<h2><strong>Bitcoin Could Catch Investors Looking For “Significant Price Appreciation” </strong></h2>
<strong>Bitcoin Could Catch Investors Looking For “Significant Price Appreciation” </strong>
2019-04-14 00:00:00
https://www.newsbtc.com/2019/04/13/bitcoin-btc-still-on-track-to-form-golden-cross-pattern-despite-recent-drop/
2019-04-13
2019-04-13 00:00:00
2019-04-13 19:30:38
<h2><strong>Bitcoin (BTC) Tepidly Moves to $5,100</strong></h2>
<strong>Bitcoin (BTC) Tepidly Moves to $5,100</strong>
2019-04-13 00:00:00
https://www.newsbtc.com/2019/04/13/fsa-bitcoin-exchange-japan/
2019-04-13
2019-04-13 00:00:00
2019-04-13 13:10:18
<h2>Security Threats</h2>
Security Threats
2019-04-13 00:00:00
https://www.newsbtc.com/2019/04/13/crypto-bigwigs-call-to-boycott-bitcoin-sv-amid-lawsuit-controversy/
2019-04-13
2019-04-13 00:00:00
2019-04-13 11:00:04
<h2>Bulk Lawsui

https://www.newsbtc.com/2019/04/10/analyst-bitcoin-bulls-are-back-critical-move-expected-soon/
2019-04-10
2019-04-10 00:00:00
2019-04-10 08:30:04
<h2>Major Similarities From 2015 Trend Reversal</h2>
Major Similarities From 2015 Trend Reversal
2019-04-10 00:00:00
https://www.newsbtc.com/2019/04/10/crypto-analyst-bitcoin-bull-run/
2019-04-10
2019-04-10 00:00:00
2019-04-10 00:01:29
<h2 class="p1"><span class="s1">Analyst: Bitcoin Bull Run Begins After Break of $5,260</span></h2>
<span class="s1">Analyst: Bitcoin Bull Run Begins After Break of $5,260</span>
2019-04-10 00:00:00
https://www.newsbtc.com/2019/04/10/bitcoin-finds-stability-above-5200-as-analysts-claim-btc-is-likely-to-continue-surging/
2019-04-10
2019-04-10 00:00:00
2019-04-10 00:00:51
<h2><strong>Bitcoin (BTC) Stable Above $5,200</strong><strong> </strong></h2>
<strong>Bitcoin (BTC) Stable Above $5,200</strong>
<strong> </strong>
2019-04-10 00:00:00
https://www.newsbtc.com/2019/04/09/paypal-bitcoin/
2019-04-09
2019-04-09 00:00

https://www.newsbtc.com/2019/04/05/saudi-arabia-pushing-usd-to-the-brink-is-good-for-bitcoin-heres-why/
2019-04-05
2019-04-05 00:00:00
2019-04-05 15:02:37
<h2>Dollar Diplomacy in Shambles</h2>
Dollar Diplomacy in Shambles
2019-04-05 00:00:00
https://www.newsbtc.com/2019/04/05/tether-volume-surpasses-bitcoin-at-18-billion-should-crypto-investors-take-it-seriously/
2019-04-05
2019-04-05 00:00:00
2019-04-05 13:10:43
<h2 class="title small">Related News</h2>
Related News
2019-04-05 00:00:00
https://www.newsbtc.com/2019/04/05/crypto-technicians-bullish-bitcoin-chain-data-bearish-story/
2019-04-05
2019-04-05 00:00:00
2019-04-05 10:00:16
<h2>Prepare For The Worst? On-Chain Bitcoin Data Signals Pullback</h2>
Prepare For The Worst? On-Chain Bitcoin Data Signals Pullback
2019-04-05 00:00:00
https://www.newsbtc.com/2019/04/05/bitcoin-accumulation-could-continue-until-july-if-previous-patterns-repeat/
2019-04-05
2019-04-05 00:00:00
2019-04-05 08:30:34
<h2>200 Days of Accumulation</h2>
200 Days of 

https://www.newsbtc.com/2019/04/02/bitcoin-briefly-over-5000-where-to-experts-see-it-going-in-near-term/
2019-04-02
2019-04-02 00:00:00
2019-04-02 13:00:51
<h2><strong>Short-Term Bearish, Long-Term Bullish On Bitcoin</strong></h2>
<strong>Short-Term Bearish, Long-Term Bullish On Bitcoin</strong>
2019-04-02 00:00:00
https://www.newsbtc.com/2019/04/02/bitcoin-rally-federal-reserve-dovish/
2019-04-02
2019-04-02 00:00:00
2019-04-02 11:30:07
<h2><strong>Bitcoin Has Found Its Identity Of Gold 2.0</strong></h2>
<strong>Bitcoin Has Found Its Identity Of Gold 2.0</strong>
2019-04-02 00:00:00
https://www.newsbtc.com/2019/04/02/bitcoin-cash-is-more-widely-accepted-than-bitcoin-in-japan-why-it-doesnt-matter/
2019-04-02
2019-04-02 00:00:00
2019-04-02 10:00:13
<h2 class="title small">Related News</h2>
Related News
2019-04-02 00:00:00
https://www.newsbtc.com/2019/04/02/crypto-trading-volume-nears-peak-levels-have-the-bulls-returned/
2019-04-02
2019-04-02 00:00:00
2019-04-02 09:00:21
<h2>Volume Hits $

RemoteDisconnected: Remote end closed connection without response

In [52]:
resultPd.to_csv("newsbtc_data.csv", encoding='utf-8', index=False)